In [3]:
# Fixing all missing imports and definitions for a Colab-compatible, self-contained suite

# Required imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sympy import primerange
from IPython.display import display

# Core constants
phi_f = (1 + math.sqrt(5)) / 2
alpha_inv = 137.035999
fine_structure_decimal = 0.0072973525693

# Utility: digit match score
def digit_match_score(a, b, window=6):
    a_str = f"{a:.16f}".replace('.', '')
    b_str = f"{b:.16f}".replace('.', '')
    max_match = 0
    for i in range(len(a_str) - window):
        a_sub = a_str[i:i + window]
        for j in range(len(b_str) - window):
            b_sub = b_str[j:j + window]
            score = sum(x == y for x, y in zip(a_sub, b_sub))
            max_match = max(max_match, score)
    return max_match

# Step 1: Test golden-damped zeta variants
def golden_damped_zeta_variants(N=50):
    primes = list(primerange(1, 1000))[:N]
    log_phi = math.log(phi_f)

    zeta_variants = {'phi^n': [], 'n^phi': [], 'n*log(phi)': []}

    for i, p in enumerate(primes):
        try:
            n = i + 1
            log_p = math.log(p)

            # Variant 1: phi^n
            exp1 = math.exp(n * log_phi)
            log_term1 = -exp1 * log_p
            term1 = math.exp(log_term1) if log_term1 > -700 else 0

            # Variant 2: n^phi
            exp2 = math.exp(phi_f * math.log(n))
            log_term2 = -exp2 * log_p
            term2 = math.exp(log_term2) if log_term2 > -700 else 0

            # Variant 3: n * log(phi)
            exp3 = n * log_phi
            log_term3 = -exp3 * log_p
            term3 = math.exp(log_term3) if log_term3 > -700 else 0

            zeta_variants['phi^n'].append(term1)
            zeta_variants['n^phi'].append(term2)
            zeta_variants['n*log(phi)'].append(term3)

        except Exception:
            zeta_variants['phi^n'].append(0)
            zeta_variants['n^phi'].append(0)
            zeta_variants['n*log(phi)'].append(0)

    results = {}
    for key, values in zeta_variants.items():
        zeta_val = sum(values)
        scaled = alpha_inv * zeta_val
        results[key] = {
            'zeta': zeta_val,
            'scaled': scaled
        }
    return results

# Step 2: Symbolic rail entropy test
def symbolic_rail_entropy(N=1000):
    rail_thresholds = [phi_f**-3, phi_f**-2, phi_f**-1]
    x = 0.314159  # initial seed
    rail_sequence = []
    for _ in range(N):
        x = (2 * x) % 1
        if x < rail_thresholds[0]:
            rail_sequence.append(0)
        elif x < rail_thresholds[1]:
            rail_sequence.append(1)
        elif x < rail_thresholds[2]:
            rail_sequence.append(2)
        else:
            rail_sequence.append(3)
    counts = [rail_sequence.count(i) for i in range(4)]
    probs = [c / N for c in counts if c > 0]
    entropy = -sum(p * math.log2(p) for p in probs)
    return entropy, rail_sequence

# Step 3: Digit match convergence curve
def golden_damped_zeta_safe(N=50, log_term_cutoff=-700):
    primes = list(primerange(1, 1000))[:N]
    log_phi = math.log(phi_f)
    terms = []

    for i, p in enumerate(primes):
        try:
            if p == 1:
                continue
            exponent = math.exp((i + 1) * log_phi)
            log_term = -exponent * math.log(p)
            if log_term < log_term_cutoff:
                terms.append(0)
            else:
                terms.append(math.exp(log_term))
        except (OverflowError, ValueError):
            terms.append(0)

    zeta_phi = sum(terms)
    scaled = alpha_inv * zeta_phi
    return zeta_phi, scaled

def zeta_digit_match_curve(N_values, constant):
    scores = []
    for N in N_values:
        try:
            _, scaled = golden_damped_zeta_safe(N)
            score = digit_match_score(scaled, constant)
            scores.append(score)
        except Exception:
            scores.append(0)
    return scores

# Run all core tests
variant_results = golden_damped_zeta_variants(N=50)
entropy_val, rail_sequence = symbolic_rail_entropy(N=1000)
match_curve = zeta_digit_match_curve(range(10, 51), fine_structure_decimal)

# Format variant result table
zeta_variant_df = pd.DataFrame.from_dict(variant_results, orient='index')
zeta_variant_df['MatchScore_alpha'] = zeta_variant_df['scaled'].apply(lambda x: digit_match_score(x, fine_structure_decimal))

# Display
display(zeta_variant_df)
print(f"\nSymbolic Rail Entropy (N=1000): {entropy_val:.4f}")


,zeta,scaled,MatchScore_alpha
phi^n,0.383223,52.515375,5
n^phi,0.534385,73.230021,3
n*log(phi),1.189130,162.953558,4



Symbolic Rail Entropy (N=1000): 0.3312
